In [1]:
import os
import math
import cv2
import dlib
import multiprocessing

# Cut out the animal's face and save the resulting image

In [2]:
def process_image(img_path, input_dir, output_dir):
    try:
        detector = dlib.cnn_face_detection_model_v1('dlib_models/dogHeadDetector.dat')
        filename, ext = os.path.splitext(os.path.basename(img_path))
        img = cv2.imread(img_path)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        
        dets = detector(img, upsample_num_times=1)
        
        for i, d in enumerate(dets):

            x1, y1 = d.rect.left(), d.rect.top()
            x2, y2 = d.rect.right(), d.rect.bottom()
            
            roi = img[y1:y2, x1:x2]
            
            output_subdir = os.path.join(output_dir, os.path.relpath(os.path.dirname(img_path), input_dir))
            os.makedirs(output_subdir, exist_ok=True)
            output_filename = f"{filename}_face{i+1}.jpg"
            output_path = os.path.join(output_subdir, output_filename)
            cv2.imwrite(output_path, cv2.cvtColor(roi, cv2.COLOR_RGB2BGR))
    except Exception as e:
        print(f"Error in {img_path}: {str(e)}")

def process_images_in_directory(input_dir, output_dir):
    with multiprocessing.Pool() as pool:
        image_paths = []
        for root, dirs, files in os.walk(input_dir):
            for file in files:
                if file.lower().endswith(('.png', '.jpg', '.jpeg', '.gif', '.bmp')):
                    img_path = os.path.join(root, file)
                    image_paths.append(img_path)

        pool.starmap(process_image, [(img_path, input_dir, output_dir) for img_path in image_paths])

In [3]:
#Raw images processing
input_directory = "images"
output_directory = "images_processed"
process_images_in_directory(input_directory, output_directory)

## Train and Test

In [4]:
import os
import shutil
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from sklearn.model_selection import train_test_split
from shutil import copyfile

2023-09-26 00:45:03.423776: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-09-26 00:45:03.425664: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-09-26 00:45:03.455381: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-09-26 00:45:03.455937: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-09-26 00:45:04.146612: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Co

In [5]:
src_dir = 'images_processed'
train_dir = 'images_processed/train'
test_dir = 'images_processed/test'

In [6]:
def clean_old_test_train(path):
    if os.path.exists(path):
        shutil.rmtree(path)  # Use rmtree to remove a directory and it's content
            print(f"Directory '{path}' successfully removed.")
    else:
        print(f"Directory '{path}' does not exist.")

In [7]:
clean_old_test_train(train_dir)
clean_old_test_train(test_dir)

Diretório 'images_processed/train' não existe.
Diretório 'images_processed/test' não existe.


## Data augmentation

In [8]:
datagen = ImageDataGenerator(rescale=1./255,
                                   rotation_range=20,
                                   width_shift_range=0.1,
                                   height_shift_range=0.1,
                                   shear_range=0.1,
                                   zoom_range=0.1,
                                   horizontal_flip=True,
                                   fill_mode='nearest')

In [9]:
def generate_and_save_images(file):
    src_file = os.path.join(src_label_dir, file)
    dst_file = os.path.join(train_label_dir, file)
    
    if not os.path.isfile(src_file):
        return
    
    img = load_img(src_file)
    img = img.resize((250, 250))
    img.save(dst_file)
    
    x = img_to_array(img)
    x = x.reshape((1,) + x.shape)

    i = 0
    for batch in datagen.flow(x, batch_size=1,
                              save_to_dir=train_label_dir, save_prefix='aug', save_format='jpeg'):
        i += 1
        if i > 10:
            break

In [10]:
labels = os.listdir(src_dir)  #Obter as labels a partir dos subdiretórios

for label in labels:
    src_label_dir = os.path.join(src_dir, label)
    train_label_dir = os.path.join(train_dir, label)
    test_label_dir = os.path.join(test_dir, label)
    
    os.makedirs(train_label_dir, exist_ok=True)
    os.makedirs(test_label_dir, exist_ok=True)
    
    files = os.listdir(src_label_dir)
    train_files, test_files = train_test_split(files, test_size=0.2, random_state=42)
    
    with multiprocessing.Pool() as p:
        p.map(generate_and_save_images, train_files)
    
    for file in test_files:
        src_file = os.path.join(src_label_dir, file)
        dst_file = os.path.join(test_label_dir, file)
        copyfile(src_file, dst_file)

In [11]:
train_datagen = ImageDataGenerator(rescale=1./255)

In [12]:
train_generator = train_datagen.flow_from_directory(train_dir,
                                                    target_size=(250, 250),
                                                    batch_size=32,
                                                    class_mode='categorical')

Found 8639 images belonging to 4 classes.


In [13]:
test_datagen = ImageDataGenerator(rescale=1./255)

In [14]:
test_generator = test_datagen.flow_from_directory(test_dir,
                                                  target_size=(250, 250),
                                                  batch_size=32,
                                                  class_mode='categorical')

Found 949 images belonging to 4 classes.


## DenseNet 121 - With LR decay

In [15]:
from tensorflow.keras.applications import DenseNet121
from keras.callbacks import EarlyStopping
import tensorflow as tf

In [16]:
def create_and_train_model_densenet_121(train_generator, test_generator, num_epochs=30, initial_learning_rate=0.001, final_learning_rate=1e-5):
    
    #Callback EarlyStopping
    early_stopping = EarlyStopping(monitor='val_accuracy',
                                   patience=15,
                                   verbose=1,
                                   restore_best_weights=True)
    
    densenet_base = tf.keras.applications.DenseNet121(weights='imagenet',
                                                      include_top=False,
                                                      input_shape=(250, 250, 3))
    
    #Freeze the layers of DenseNet121
    densenet_base.trainable = False
    
    #Generation of the steps value
    total_training_examples = int(train_generator.samples)
    batch_size = train_generator.batch_size
    steps_per_epoch = math.ceil(total_training_examples / batch_size)
    validation_steps = test_generator.samples // test_generator.batch_size
    

    model_densenet = tf.keras.models.Sequential([
        densenet_base,
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(512, activation='relu'),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(4, activation='softmax')
    ])
    
    #Decay learning rate
    decay_rate = (final_learning_rate / initial_learning_rate) ** (1 / num_epochs)
    
    #Learning rate scheduling function
    def lr_schedule(epoch):
        current_learning_rate = initial_learning_rate * (decay_rate ** epoch)
        return current_learning_rate
    
    model_densenet.compile(loss='categorical_crossentropy',
                           optimizer=tf.keras.optimizers.Adam(learning_rate=initial_learning_rate),
                           metrics=['accuracy'])
    
    #lr_schedule function to schedule the learning rate
    lr_callback = tf.keras.callbacks.LearningRateScheduler(lr_schedule)

    history_densenet = model_densenet.fit(train_generator,
                                          steps_per_epoch=steps_per_epoch,
                                          epochs=num_epochs,
                                          validation_data=test_generator,
                                          validation_steps=validation_steps,
                                          callbacks=[early_stopping, lr_callback])
    
    return model_densenet, history_densenet

In [17]:
model_densenet_121, history_densenet_121 = create_and_train_model_densenet_121(train_generator, test_generator, num_epochs=100)

2023-09-26 00:47:16.637687: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-09-26 00:47:16.638966: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1960] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


Epoch 1/100
270/270 [==============================] - 369s 1s/step - loss: 4.0865 - accuracy: 0.5787 - val_loss: 0.8466 - val_accuracy: 0.6681 - lr: 0.0010
Epoch 2/100
270/270 [==============================] - 351s 1s/step - loss: 0.7707 - accuracy: 0.6927 - val_loss: 0.8891 - val_accuracy: 0.6746 - lr: 9.5499e-04
Epoch 3/100
270/270 [==============================] - 351s 1s/step - loss: 0.6970 - accuracy: 0.7223 - val_loss: 0.8234 - val_accuracy: 0.6853 - lr: 9.1201e-04
Epoch 4/100
270/270 [==============================] - 352s 1s/step - loss: 0.6375 - accuracy: 0.7468 - val_loss: 0.8959 - val_accuracy: 0.6466 - lr: 8.7096e-04
Epoch 5/100
270/270 [==============================] - 352s 1s/step - loss: 0.5532 - accuracy: 0.7789 - val_loss: 0.8939 - val_accuracy: 0.6724 - lr: 8.3176e-04
Epoch 6/100
270/270 [==============================] - 353s 1s/step - loss: 0.5244 - accuracy: 0.7886 - val_loss: 0.9634 - val_accuracy: 0.6724 - lr: 7.9433e-04
Epoch 7/100
270/270 [=================

In [18]:
test_generator.samples

949

In [19]:
test_loss_densenet_121, test_acc_densenet_121 = model_densenet_121.evaluate(test_generator, verbose=2)
print('\nTest accuracy:', test_acc_densenet_121)

30/30 - 32s - loss: 0.9631 - accuracy: 0.6934 - 32s/epoch - 1s/step

Test accuracy: 0.6933614611625671


## DenseNet 201 - With LR decay

In [20]:
from tensorflow.keras.applications import DenseNet201
from keras.callbacks import EarlyStopping
import tensorflow as tf

In [21]:
def create_and_train_model_densenet_201(train_generator, test_generator, num_epochs=30, initial_learning_rate=0.001, final_learning_rate=1e-5):
    
    #Callback EarlyStopping
    early_stopping = EarlyStopping(monitor='val_accuracy',
                                  patience=15,
                                  verbose=1,
                                  restore_best_weights=True)

    
    densenet_base = tf.keras.applications.DenseNet201(weights='imagenet',
                                                      include_top=False,
                                                      input_shape=(250, 250, 3))
    
    #Freeze the layers of DenseNet201
    densenet_base.trainable = False

    #Geração do valor de steps
    total_training_examples = train_generator.samples
    batch_size = train_generator.batch_size
    steps_per_epoch = total_training_examples / batch_size
    validation_steps = test_generator.samples // test_generator.batch_size

    model_densenet = tf.keras.models.Sequential([
        densenet_base,
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(512, activation='relu'),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(4, activation='softmax')
    ])
    
    #Decay learning rate
    decay_rate = (final_learning_rate / initial_learning_rate) ** (1 / num_epochs)
    
    #Learning rate scheduling function
    def lr_schedule(epoch):
        current_learning_rate = initial_learning_rate * (decay_rate ** epoch)
        return current_learning_rate

    model_densenet.compile(loss='categorical_crossentropy',
                           optimizer=tf.keras.optimizers.Adam(learning_rate=initial_learning_rate),  # Use learning_rate
                           metrics=['accuracy'])
    
    #lr_schedule function to schedule the learning rate
    lr_callback = tf.keras.callbacks.LearningRateScheduler(lr_schedule)

    history_densenet = model_densenet.fit(train_generator,
                                          steps_per_epoch=steps_per_epoch,
                                          epochs=num_epochs,
                                          validation_data=test_generator,
                                          validation_steps=validation_steps,
                                          callbacks=[early_stopping, lr_callback])
    
    return model_densenet, history_densenet

In [22]:
model_densenet_201, history_densenet_201 = create_and_train_model_densenet_201(train_generator, test_generator, num_epochs=100)

Epoch 1/100
269/269 [==============================] - 557s 2s/step - loss: 3.2795 - accuracy: 0.6181 - val_loss: 0.9156 - val_accuracy: 0.6121 - lr: 0.0010
Epoch 2/100
269/269 [==============================] - 550s 2s/step - loss: 0.7102 - accuracy: 0.7154 - val_loss: 0.7776 - val_accuracy: 0.6875 - lr: 9.5499e-04
Epoch 3/100
269/269 [==============================] - 549s 2s/step - loss: 0.5700 - accuracy: 0.7746 - val_loss: 0.8019 - val_accuracy: 0.7080 - lr: 9.1201e-04
Epoch 4/100
269/269 [==============================] - 550s 2s/step - loss: 0.4831 - accuracy: 0.8084 - val_loss: 0.7792 - val_accuracy: 0.7091 - lr: 8.7096e-04
Epoch 5/100
269/269 [==============================] - 549s 2s/step - loss: 0.4431 - accuracy: 0.8209 - val_loss: 0.8405 - val_accuracy: 0.6918 - lr: 8.3176e-04
Epoch 6/100
269/269 [==============================] - 549s 2s/step - loss: 0.3945 - accuracy: 0.8419 - val_loss: 0.8537 - val_accuracy: 0.7155 - lr: 7.9433e-04
Epoch 7/100
269/269 [=================

In [23]:
test_loss_densenet_201, test_acc_densenet_201 = model_densenet_201.evaluate(test_generator, verbose=2)
print('\nTest accuracy:', test_acc_densenet_201)

30/30 - 50s - loss: 1.0546 - accuracy: 0.7281 - 50s/epoch - 2s/step

Test accuracy: 0.7281348705291748


## Save and load models

In [24]:
def save_model(model, filename):
    model.save('cnn_models/'+filename)

def load_model(filename):
    model = load_model('cnn_models/'+filename)
    return model

In [25]:
save_model(model_densenet_121, 'model_densenet_121.h5')
save_model(model_densenet_201, 'model_densenet_201.h5')

/home/leonardocapellaro/anaconda3/lib/python3.9/site-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


## Predict image classes

In [26]:
def process_image_predict(img_path, target_size=(250, 250)):
    try:
        detector = dlib.cnn_face_detection_model_v1('dlib_models/dogHeadDetector.dat')
        img = cv2.imread(img_path)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

        dets = detector(img, upsample_num_times=1)

        # Se nenhuma detecção de face for encontrada, retorne uma imagem em branco
        if len(dets) == 0:
            print("No faces were found in the image")
            return np.zeros((target_size[0], target_size[1], 3), dtype=np.uint8)

        # Desenhe retângulos nas detecções
        for i, d in enumerate(dets):
            x1, y1 = d.rect.left(), d.rect.top()
            x2, y2 = d.rect.right(), d.rect.bottom()
            cv2.rectangle(img, (x1, y1), (x2, y2), (0, 255, 0), 2)

        # Converta a imagem resultante de volta para BGR para salvar ou exibir
        img_result = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)

        # Redimensione a imagem resultante para o tamanho desejado
        img_result = cv2.resize(img_result, target_size)

        return img_result

    except Exception as e:
        print(f"Error in image {img_path}: {str(e)}")

In [27]:
import tensorflow as tf
from tensorflow.keras.preprocessing import image
import numpy as np

def image_class_predict(model, image_path):
    classes = ["angry", "happy", "sad"]
    # Carregue e pré-processe a imagem de entrada
    img = process_image_predict(image_path)
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array /= 255.0  # Normalização (se necessário)

    # Faça a previsão
    predictions = model.predict(img_array)

    # Obtenha a classe prevista
    class_index = np.argmax(predictions)

    return class_index, classes[class_index]

## Result using Densenet 121

In [ ]:
#Uncomment to load the downloaded model in cnn_models folder
#model_densenet_121 = load_model('model_densenet_121.h5')

In [28]:
image = 'example.jpeg'
predicted_class_index, predicted_class = image_class_predict(model_densenet_121, image)

print(f'The image is from the class: {predicted_class}')

1/1 [==============================] - 2s 2s/step
A imagem é da classe: happy


## Result using Densenet 201

In [ ]:
#Uncomment to load the downloaded model in cnn_models folder
#model_densenet_201 = load_model('model_densenet_201.h5')

In [29]:
imagem = 'example.jpeg'
predicted_class_index, predicted_class = image_class_predict(model_densenet_201, imagem)

print(f'The image is from the class: {predicted_class}')

1/1 [==============================] - 3s 3s/step
A imagem é da classe: happy
